In [1]:
# verified

import json
import sys
import os

from os.path import join

sys.path.append('../..')
from squad_tools import load_dict
from my_tokenize import tokenize

In [6]:
squad_path = '/pio/data/data/squad/'

with open(join(squad_path, 'negative_samples', 'train.wiki.pos.json')) as f:
    wiki_data_pos = json.load(f)
    
glove_path = '/pio/data/data/glove_vec/6B/glove.6B.300d.txt'
squad_preprocced_path = join(squad_path, 'test')
out_path = join(squad_preprocced_path, 'wiki_pos')

wordlist = load_dict(glove_path)
w_to_i = {w : i for i, w in enumerate(wordlist)}

NAW_tok = u'<not_a_word>'
NAW_idx = wordlist.index(NAW_tok)
NAW_chars = [1, 3, 2]

lower = True
def lower_if_needed(s):
    if lower:
        s = s.lower()
    return s

In [19]:
wiki_data_pos_prep = []

for d in wiki_data_pos:
    q = tokenize(lower_if_needed(d[1]))
    x = tokenize(lower_if_needed(d[3]))
    ans = [list(range(d[4], d[4] + len(tokenize(d[2]))))]
    x.append(NAW_tok)
    wiki_data_pos_prep.append([ans, q, x])
    
wiki_data_pos = wiki_data_pos_prep

In [23]:
def get_word_nums(s):
    return [w_to_i.get(w, 0) for w in s]

wiki_pos_words = []
for a, q, x in wiki_data_pos:
    q_num = get_word_nums(q)
    x_num = get_word_nums(x)
    wiki_pos_words.append([a, q_num, x_num])

In [27]:
def make_bin_feats(sample):
    q, x = sample
    qset = set(q)
    return [w in qset for w in x]

wiki_pos_bin_feats = []

for _, q, c in wiki_data_pos:
    wiki_pos_bin_feats.append(make_bin_feats([q, c]))

In [28]:
wiki_pos_ascii_chars = []

chars = [unichr(i) for i in range(128)]
c_to_i = {chars[i] : i for i in range(128)}

def get_char_nums_for_word(w):
    return [1] + [c_to_i.get(c, 0) for c in w] + [2]

for _, q, x in wiki_data_pos:
    q_char = map(get_char_nums_for_word, q)
    x_char = map(get_char_nums_for_word, x[:-1]) + [[1, 3, 2]]
    wiki_pos_ascii_chars.append([q_char, x_char])

In [29]:
if not os.path.exists(out_path):
    os.makedirs(out_path)
    
with open(join(out_path, 'train_words.json'), 'w') as f:
    json.dump(wiki_pos_words, f)

with open(join(out_path, 'train_bin_feats.json'), 'w') as f:
    json.dump(wiki_pos_bin_feats, f)
    
with open(join(out_path, 'train_char_ascii.json'), 'w') as f:
    json.dump(wiki_pos_ascii_chars, f)